# Revised case normalization for Hirslanden Zürich 2017

Notes:

- 17 of 17 revised DtoD cases were joined with the BFS cases from the DB.



In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group, format_for_grouper_one_case, format_for_grouper
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(os.path.join(ROOT_DIR, 'raw_data/HI-Zurich.xlsx'),
                     'Hirslanden Zurich', '2017', 'Änderungen_Hirslanden_2017')

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Zurich.xlsx', hospital_name_db='Hirslanden Zurich', year='2017', sheet='Änderungen_Hirslanden_2017')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-10 13:47:11.623 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:35 - Read 128 cases for Hirslanden Zurich 2017
2022-11-10 13:47:11.628 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:60 - TYPES:
datum                                                string
tranche                                              string
case_id                                              object
patient_id                                           object
gender                                               string
age_years                                             int64
bfs_code                                             string
duration_of_stay                                      int64
pflegetage neu                                       string
pccl                                                  int64
pccl neu                                             string
old_pd                                               string
primary

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,41230054,E8CD6759509644DE,M,78,11,41230054,C341,C341,M200,3,E06C,[J939],[],[],[]
1,41194118,41309F148FEDBFD2,W,39,7,41194118,O335,O335,M300,3,O01E,[K560],[],[],[]
2,41230059,7E79B28620A3A8E3,M,84,8,41230059,C61,C61,M200,3,M09B,[C795],[],[],[]
3,41231720,FBB0408939EC3E8B,M,81,6,41231720,J4402,J4402,M100,3,E65C,[E1191],[E1190],[],[]
4,41243610,32E2ED09CA696444,W,82,7,41243610,J4412,J4412,M100,3,E65C,[J81],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,41249123,9F60C5E2FF49ADFA,M,72,9,41249123,C671,C671,M200,2,L18B,[U804],[],[],[]
124,41323252,473038E4266105E6,W,80,8,41323252,N390,N390,M100,3,L06B,[U804],[],[],[]
125,41340499,38988935DC3B2582,M,57,7,41340499,N10,N10,M100,3,L63D,[U804],[U81],[],[]
126,41347960,4CA999AAFA0018CC,W,76,3,41347960,J4481,J4480,M100,0,E65C,[],[],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases.head()

2022-11-10 13:47:13.202 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:90 - Read 19047 rows from the DB, for the hospital 'Hirslanden Zurich' in 2017


,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
0,45537,41418386,M4806,"[G553, M4785, I1091, I7021, J4482, ...]",8108::20170704,"[816220::20170704, 785035::20170704, 805110::20170704, 845110::20170704, 784987::20170704, ...]",W,79,0,0,16,0,01,2017-07-03,0,00,2017-07-19
1,45602,41243205,N390,"[M9326, G301, F001, R470]",992212::20170109,[948X40::20170109],W,80,0,0,10,0,01,2017-01-09,0,00,2017-01-19
2,45634,41486629,I714,"[T828, Y828, I1090, E780, T818, ...]",397114::20170831,"[39783G::20170831, 399110:B:20170831, 884211::20170831, 395011:L:20170831, 399010:L:20170831, ...]",M,68,0,0,5,0,01,2017-08-30,0,00,2017-09-04
3,45892,41385331,I5001,"[I5013, I2728, Z950, E876, N183, ...]",992212::20170515,"[890A32::20170515, 8945::20170515]",W,92,0,0,17,0,01,2017-05-15,0,00,2017-06-01
4,46427,41577513,E222,"[Y579, E871, R400, R470, N40, ...]",99B712::20171114,"[891499::20171114, 0331::20171114, 5794::20171115, 889110::20171116]",M,82,0,0,13,0,11,2017-11-14,0,00,2017-11-27


In [5]:
if unmatched.shape[0] > 0:
    unmatched

In [6]:
revised_cases.loc[106]

aimedic_id                                                61304
case_id                                                41604407
primary_diagnosis                                         J4400
secondary_diagnoses       [J4400, D638, J9610, I1090, N40, ...]
primary_procedure                                              
secondary_procedures                                         []
gender                                                        M
age_years                                                    72
age_days                                                      0
gestation_age                                                 0
duration_of_stay                                              5
ventilation_hours                                             0
grouper_admission_type                                       01
admission_date                                       2017-12-06
admission_weight                                              0
grouper_discharge_type                  

In [7]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-10 13:47:13.629 | INFO     | src.revised_case_normalization.notebook_functions.group:group:83 - Grouping 126 cases ...
2022-11-10 13:47:14.850 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:88 - Grouped 126 cases into: 126 revisions, 1224 diagnoses rows, 634 procedure rows


In [8]:
procedures_update

,aimedic_id,code,side,date,is_grouper_relevant,is_primary
0,45537,8108,,2017-07-04,False,False
1,45537,816220,,2017-07-04,True,False
2,45537,785035,,2017-07-04,False,False
3,45537,805110,,2017-07-04,False,False
4,45537,845110,,2017-07-04,False,False
...,...,...,...,...,...,...
629,64444,862B2D,,2017-10-23,False,False
630,64444,889416,,2017-10-22,False,False
631,64444,862B2D,,2017-10-24,False,False
632,64444,862B2D,,2017-10-25,False,False


In [9]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,45537,I09D,2.58,2.58,3,2022-12-31
1,45602,L63D,0.67,0.67,3,2022-12-31
2,45634,F34A,3.74,3.74,4,2022-12-31
3,45892,F62B,1.38,1.38,4,2022-12-31
4,46427,K64A,1.37,1.37,4,2022-12-31
...,...,...,...,...,...,...
121,63548,F71B,0.74,0.74,3,2022-12-31
122,63919,I10A,2.67,1.86,4,2022-12-31
123,63942,G48B,0.90,0.90,3,2022-12-31
124,64168,B02C,3.98,3.36,4,2022-12-31


In [10]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-10 13:47:14.887 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:249 - Trying to insert 126 cases into the 'Revisions' table ...
2022-11-10 13:47:15.688 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:275 - Deleted 126 rows from the "Revisions" table, which is about to be updated
2022-11-10 13:47:15.897 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:286 - Inserted 126 cases into the 'Revisions' table
2022-11-10 13:47:15.898 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:298 - Trying to insert 1224 rows into the 'Diagnoses' table ...
2022-11-10 13:47:18.269 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:336 - Inserted 1224 rows into the 'Diagnoses' table
2022-11-10 13:47:18.272 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:347 - Trying to insert 634 rows into the 'Procedure